<a href="https://colab.research.google.com/github/Rathod-shubhamm/Book-recommendation-engine/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-10-01 07:31:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-10-01 07:31:31 (206 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:

print(df_ratings.shape)
print(df_books.shape)



(1149780, 3)
(271379, 3)


In [5]:
book_rating = df_ratings["isbn"].value_counts()
popular_books = book_rating[book_rating >= 100].index
df_ratings_new = df_ratings[df_ratings['isbn'].isin(popular_books)]

user_rating = df_ratings['user'].value_counts()
active_users = user_rating[user_rating >= 200].index
df_ratings_new = df_ratings_new[df_ratings_new['user'].isin(active_users)]

In [6]:
df_ratings_new.shape
df = pd.merge(df_ratings_new,df_books, on='isbn')
df.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49517 entries, 0 to 49516
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49517 non-null  int32  
 1   isbn    49517 non-null  object 
 2   rating  49517 non-null  float32
 3   title   49517 non-null  object 
 4   author  49517 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.5+ MB


In [8]:
duplicates = df.duplicated(subset= ['user', 'title'])
duplicate_rows = df[duplicates]
print(len(duplicate_rows))

381


In [9]:
df.drop_duplicates(subset =['user', 'title'], inplace = True)

In [10]:
df_matrix = df.pivot(index ='title', columns = 'user' , values= "rating"  )
df_matrix.fillna(0, inplace  =  True )
df_matrix.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_matrix.shape

(673, 888)

In [12]:
model=NearestNeighbors(n_neighbors=6,metric='cosine',algorithm='brute')
model.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [13]:
distance  , indice =  model.kneighbors([df_matrix.loc["Where the Heart Is (Oprah's Book Club (Paperback))"].values],n_neighbors =6)
df2 =pd.DataFrame({"title" : df_matrix.iloc[indice[0]].index.values,
                  "distance" : distance[0]})
df2.sort_values(by = "distance" ,ascending = False)[:4]

,title,distance
5,I'll Be Seeing You,0.801621
4,The Weight of Water,0.770858
3,The Surgeon,0.769941
2,I Know This Much Is True,0.767708


In [18]:
# Create a DataFrame with the recommended book titles and their distances
def get_recommends(book, n=5):
    distances, indices = model.kneighbors([df_matrix.loc[book].values], n_neighbors=n+1)


    data = pd.DataFrame({
        "title": df_books.iloc[indices[0]]['title'].values,  # Get titles from df_books
        'distance': distances[0]  # Corresponding distances
    })
    data_sorted = data.sort_values(by="distance", ascending=False).iloc[1:]  # Exclude the first result (the book itself)
    recommendations = data_sorted[['title', 'distance']].values.tolist()
    result = [book, recommendations]

    return result

In [19]:

books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77, ]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074]]]
You passed the challenge! 🎉🎉🎉🎉🎉
